In [ ]:
!git clone https://github.com/atharva1707/Behavioral.git


Cloning into 'Behavioral'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 10), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (32/32), 6.81 MiB | 6.29 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [ ]:
%run /content/Behavioral/load_data.ipynb

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = load_all_data()

In [ ]:
print(data[2])

       SUBJECT  SESSION  RUN  TRIAL  73KID  10KID        TIME  ISOLD  \
0            3        1    1      1  46003    626    0.399438      0   
1            3        1    1      2  19257   5013    0.399484      0   
2            3        1    1      3  36386   4850    0.399531      0   
3            3        1    1      4  44343   8823    0.399577      0   
4            3        1    1      5  62685   9538    0.399623      0   
...        ...      ...  ...    ...    ...    ...         ...    ...   
23995        3       32   12     58  66372   7880  256.724513      1   
23996        3       32   12     59  50764   5270  256.724560      1   
23997        3       32   12     60  56937   4082  256.724606      1   
23998        3       32   12     61  28438   2537  256.724652      0   
23999        3       32   12     62  42959   1804  256.724699      1   

       ISCORRECT           RT  CHANGEMIND  MEMORYRECENT  MEMORYFIRST  \
0            1.0  1570.914331         0.0          -1.0        

In [ ]:

def get_temporal_df(data_all):
  # list to store all entries
  pos_data_list = []
  neg_data_list = []
  # for all subjects
  for data in data_all:
    print(data)
    # iterate all rows
    for index, row in data.iterrows():
      # identify correctly recognised images being repeated for the first time
      if  row["freq"] == 2:
        # Check next 5 rows (excluding current row)
        for i in range(1, 6):
          index3 = index
          index4 = index + i
          # Reached the end of dataframe
          if index4 >= len(data):
            break

          # next 5 images must be within the same run
          if data.loc[index3,"RUN"] != data.loc[index4,"RUN"]:
            continue

          # both the images should have a previous occurence
          if data.loc[index3,"MEMORYRECENT"] == -1 or data.loc[index4,"MEMORYRECENT"] == -1:
            continue

          index2 = index3 - data.loc[index3,"MEMORYRECENT"] - 1
          index1 = index4 - data.loc[index4,"MEMORYRECENT"] - 1
          # ADDING CONDITIONS FOR TEMPORAL CONTEXT MODEL

          # INDEX1 AND INDEX4 CANT BE IN SAME SESSION (HIGH EFFECT OF SELF DISTANCE ON ACCURACY)
          if  data.loc[index4,"SESSION"] - data.loc[index1,"SESSION"] <= 0:
            continue

          if  data.loc[index4,"SESSION"] - data.loc[index1,"SESSION"] > 1:
            continue
          # INDEX1 AND INDEX2 MUST BE WITHIN THE SAME SESSION FOR LOW TEMPORAL DISTANCE
          if data.loc[index1,"SESSION"] != data.loc[index2,"SESSION"]:
            continue
          # INDEX1 AND INDEX2 MUST BE WITHIN THE SAME RUN FOR LOW TEMPORAL DISTANCE
          if index1 - index2 > 60 or index2 - index1 > 60:
            continue

          # print("HERE", data.loc[index3,"MEMORYRECENT"])

          # subject, , temp_dist, temp_run,
          LAG = data.loc[index4,"SESSION"] -  data.loc[index1,"SESSION"]
          # LAG = index4 - index1
          # TDE = data.loc[index2,"RUN"] -  data.loc[index1,"RUN"]
          TDE = index2 - index1
          SUBJECT = data.loc[0,"SUBJECT"]
          ISCORRECT = data.loc[index4,"ISCORRECT"]


          data_dict = {
            'SUBJECT': SUBJECT,
            'LAG': LAG,
            'TDE': TDE,
            'ISCORRECT': ISCORRECT,
            'INDEX1': index1,
            'INDEX2': index2,
            'INDEX3': index3,
            'INDEX4': index4,
            'TCR': row["ISCORRECT"]
        }

          # Append the dictionary to the list
          if TDE < 0:
            data_dict['TDE'] = -data_dict['TDE']
            neg_data_list.append(data_dict)
          else:
            pos_data_list.append(data_dict)

  pdf = pd.DataFrame(pos_data_list)
  ndf = pd.DataFrame(neg_data_list)
  return pdf,ndf


In [ ]:
pdf,ndf = get_temporal_df(data)
df = pdf

       SUBJECT  SESSION  RUN  TRIAL  73KID  10KID        TIME  ISOLD  \
0            1        1    1      1  46003    626    0.505082      0   
1            1        1    1      2  61883   5013    0.505128      0   
2            1        1    1      3    829   4850    0.505175      0   
3            1        1    1      4  67574   8823    0.505221      0   
4            1        1    1      5  16021   9538    0.505267      0   
...        ...      ...  ...    ...    ...    ...         ...    ...   
29995        1       40   12     58  13774   8984  262.629551      1   
29996        1       40   12     59  66768   6026  262.629597      1   
29997        1       40   12     60  53168   4841  262.629644      1   
29998        1       40   12     61   1944   7323  262.629690      1   
29999        1       40   12     62   5034   9972  262.629736      1   

       ISCORRECT           RT  CHANGEMIND  MEMORYRECENT  MEMORYFIRST  \
0            1.0   803.529781         0.0          -1.0        

In [ ]:
df = ndf

In [ ]:
print(df.shape)
print(df)
print((df["TCR"]==0).sum())

(936, 9)
     SUBJECT  LAG   TDE  ISCORRECT   INDEX1   INDEX2  INDEX3  INDEX4  TCR
0          1    1  18.0        1.0    232.0    214.0     797     798  1.0
1          1    1  42.0        1.0    699.0    657.0     890     895  1.0
2          1    1  24.0        0.0     97.0     73.0     948     953  0.0
3          1    1  59.0        1.0   2234.0   2175.0    2370    2373  1.0
4          1    1  12.0        0.0   2790.0   2778.0    3007    3009  0.0
..       ...  ...   ...        ...      ...      ...     ...     ...  ...
931        8    1  16.0        0.0  20727.0  20711.0   21011   21016  1.0
932        8    1   2.0        1.0  21651.0  21649.0   21820   21824  1.0
933        8    1  20.0        1.0  21711.0  21691.0   21875   21877  1.0
934        8    1  57.0        1.0  21656.0  21599.0   21951   21952  1.0
935        8    1  47.0        1.0  21662.0  21615.0   22295   22297  1.0

[936 rows x 9 columns]
429


In [ ]:
a = 0
b = 0
for index, row in data[7].iterrows():
  if(data[0].loc[index,"MEMORYRECENT"]) != (data[7].loc[index,"MEMORYRECENT"]) or (data[0].loc[index,"MEMORYFIRST"]) != (data[7].loc[index,"MEMORYFIRST"]):
    a = a + 1
  else:
    b = b + 1
print(a,b)


0 22500


In [ ]:
import pandas as pd
import plotly.express as px

# Get unique subjects from the DataFrame
unique_subjects = df[df["TCR"]==1]['SUBJECT'].unique()

# Plot individual scatter plots for each subject
for subject in unique_subjects:
  # Filter DataFrame for the current subject
  subject_data = df[df['SUBJECT'] == subject]

  # Create a scatter plot using Plotly Express
  color = ['red' if x == 0 else 'green' for x in subject_data['ISCORRECT']]
  fig = px.scatter(
      subject_data,
      x='TDE',
      y='LAG',
      color=color,
      title=f'Subject {subject} - Scatter Plot of LAG vs TDE',
      labels={'x': 'TDE', 'y': 'LAG'},
      color_discrete_map={'red': 'red', 'green': 'green'}
  )
  fig.update_traces(marker=dict(size=10, opacity=0.2))  # Set opacity

  # Show the plot for the current subject
  fig.show()


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

def run_mixed_effects_logit(df):
    # Convert data to a DataFrame for analysis
    results = {}
    # Group data by subject
    subject_data = {}
    for subject, subject_df in df.groupby("SUBJECT"):
        subject_data[subject] = subject_df


    # Loop through each subject
    for subject, subject_df in subject_data.items():
        sub_df = pd.DataFrame(subject_df)

        # Build mixed effects logistic regression model
        formula = 'ISCORRECT ~  TDE + TCR '
        model = smf.mixedlm(formula, sub_df)
        result = model.fit()

        # Store model result
        results[subject] = result
        print(result.summary())


    return results

In [ ]:
run_mixed_effects_logit(df)

AttributeError: 'groups' is a required keyword argument in MixedLM.from_formula

In [ ]:
# Build mixed effects logistic regression model
formula = 'ISCORRECT ~  TDE + TCR '
model = smf.mixedlm(formula, df, groups= df['SUBJECT'])
result = model.fit()
# Print model summary
print(result.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: ISCORRECT
No. Observations: 1012    Method:             REML     
No. Groups:       8       Scale:              0.2086   
Min. group size:  115     Log-Likelihood:     -663.6268
Max. group size:  136     Converged:          Yes      
Mean group size:  126.5                                
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.562    0.067  8.348 0.000  0.430  0.694
TDE          -0.002    0.001 -2.138 0.033 -0.003 -0.000
TCR           0.135    0.030  4.430 0.000  0.075  0.194
Group Var     0.028    0.035                           



In [ ]:
def count_filtered_rows(df, TCR_VALUE, TDE_START, TDE_END):
    """
    Count rows in DataFrame `df` based on specified conditions.

    Parameters:
    - df (pandas.DataFrame): Input DataFrame containing 'TCR', 'TDE', and 'ISCORRECT' columns.
    - TCR_VALUE (int or float): Value of 'TCR' to filter rows.
    - TDE_START (int or float): Start value (inclusive) of 'TDE' range to filter rows.
    - TDE_END (int or float): End value (exclusive) of 'TDE' range to filter rows.

    Returns:
    - int: Number of rows in `df` satisfying the specified conditions.
    """
    # Filter rows based on conditions
    filtered_df = df[(df['TCR'] == TCR_VALUE) &
                     (df['TDE'] >= TDE_START) &
                     (df['TDE'] < TDE_END) &
                     (df['SUBJECT'] == 1)]

    # Count the number of rows in the filtered DataFrame
    accuracy = (len(filtered_df[filtered_df["ISCORRECT"]==1])/len(filtered_df),len(filtered_df) )

    count = len(filtered_df)

    return count, accuracy

In [ ]:
# ACCURACY COMPARISION
print(count_filtered_rows(df,1,0,3))
print(count_filtered_rows(df,1,3,6))
print(count_filtered_rows(df,1,6,9))
print(count_filtered_rows(df,1,9,12))
print(count_filtered_rows(df,1,12,15))
print(count_filtered_rows(df,1,15,18))
print(count_filtered_rows(df,1,18,21))
print(count_filtered_rows(df,1,21,24))
print(count_filtered_rows(df,1,24,27))
print("_________________")
print(count_filtered_rows(df,0,0,3))
print(count_filtered_rows(df,0,3,6))
print(count_filtered_rows(df,0,6,9))
print(count_filtered_rows(df,0,9,12))
print(count_filtered_rows(df,0,12,15))
print(count_filtered_rows(df,0,15,18))
print(count_filtered_rows(df,0,18,21))
print(count_filtered_rows(df,0,21,24))
print(count_filtered_rows(df,0,24,27))


(1, (1.0, 1))
(9, (1.0, 9))
(8, (1.0, 8))
(6, (0.6666666666666666, 6))
(3, (0.3333333333333333, 3))
(2, (1.0, 2))
(4, (0.0, 4))
(5, (1.0, 5))
(6, (0.8333333333333334, 6))
_________________
(1, (1.0, 1))
(1, (1.0, 1))
(5, (0.6, 5))
(1, (1.0, 1))
(5, (0.8, 5))
(3, (0.6666666666666666, 3))
(1, (1.0, 1))
(3, (1.0, 3))
(1, (1.0, 1))
